In [14]:
import numpy as np
import matplotlib.pyplot as plt
from mpl_toolkits import mplot3d
import math
from scipy import signal
from scipy.fftpack import fft
from scipy import ndimage
%matplotlib inline

In [15]:
import os
os.chdir('C:\\Users\\hydro_leo\\Documents\\GitHub\\BROJA_2PID')
from BROJA_2PID import pid, BROJA_2PID_Exception

In [16]:
def Histo3D2Dict(P):
    PDF = dict()
    for i in range(np.size(P, 0)):
        for j in range(np.size(P, 1)):
            for k in range(np.size(P, 2)):
                PDF[(k,i,j)] = float(P[i,j,k])
    return PDF


In [22]:
def PIfunc(r, x, v, dt, window):
    negshift=window[0] # second
    posshift=window[1] # second
    shiftlen=(posshift-negshift)/dt+1
    timeshift=np.linspace(negshift,posshift,int(shiftlen))
    bitshift=np.linspace(negshift/dt,posshift/dt,int(shiftlen),dtype = 'int16')
    Information = dict()
    Information[('BROJA_2PID','SI')]=np.zeros(len(bitshift))
    Information[('BROJA_2PID','UIx')]=np.zeros(len(bitshift))
    Information[('BROJA_2PID','UIv')]=np.zeros(len(bitshift))
    Information[('BROJA_2PID','CI')]=np.zeros(len(bitshift))
    Information[('Beer','Red')]=np.zeros(len(bitshift))
    Information[('Beer','UIx')]=np.zeros(len(bitshift))
    Information[('Beer','UIv')]=np.zeros(len(bitshift))
    Information[('Beer','Syn')]=np.zeros(len(bitshift))
    
    Information[('test','SI')]=np.zeros(len(bitshift))
    Information[('test','UIx')]=np.zeros(len(bitshift))
    Information[('test','UIv')]=np.zeros(len(bitshift))
    Information[('test','CI')]=np.zeros(len(bitshift))
    Redun =np.zeros(len(bitshift))
    MIxr=np.zeros(len(bitshift))
    MIvr=np.zeros(len(bitshift))
    MIxvR=np.zeros(len(bitshift))
    parms = dict()
    parms['max_iters'] = 10
    # shifted data
    # shift>0 => y shifted to positive side
    for i in range(len(bitshift)):
        xx=[]
        vv=[]
        rr=[]
        shift=bitshift[i]
        if shift>0:
            xx=x[shift:]
            vv=v[shift:]
            rr=r[:(-1*shift)]
        elif shift==0:
            xx=x
            vv=v
            rr=r
        elif shift<0:
            xx=x[:shift]
            vv=v[:shift]
            rr=r[(-1*shift):]
        #find weight of each states by 3D histogram 
        xedges = np.append(np.unique(xx),(max(xx)+1))
        vedges = np.append(np.unique(vv),(max(vv)+1))
        redges = np.append(np.unique(rr),(max(rr)+1))
        dat = np.concatenate((xx[:,np.newaxis], vv[:,np.newaxis],rr[:,np.newaxis]), axis=1)
        N, edges = np.histogramdd(dat, bins=(xedges, vedges, redges))
        #Calculate all kinds of probability and make sure the shape of them, 0 -> x, 1 -> v, 2 -> r
        px=(np.sum(N,axis=(1,2))/np.sum(N))[:, np.newaxis, np.newaxis]
        pv=(np.sum(N,axis=(0,2))/np.sum(N))[np.newaxis, :, np.newaxis]
        pr=(np.sum(N,axis=(0,1))/np.sum(N))[np.newaxis ,np.newaxis, :]
        pxv=(np.sum(N,axis=2)/np.sum(N))[:, :, np.newaxis]
        pxr=(np.sum(N,axis=1)/np.sum(N))[:, np.newaxis, :]
        pvr=(np.sum(N,axis=0)/np.sum(N))[np.newaxis, :, :]
        pxvr=(N/np.sum(N))
        
        Information[('test','UIx')][i] = np.nansum(pxvr*np.log2(pxvr*px/pxv/pxr))/dt
        
        Information[('test','UIv')][i] = np.nansum(pxvr*np.log2(pxvr*pv/pxv/pvr))/dt
                
        PDF=Histo3D2Dict(pxvr)
        BROJA_2PID = pid(PDF, cone_solver="ECOS", output=0, **parms)
        Information[('BROJA_2PID','SI')][i]=BROJA_2PID['SI']/dt
        Information[('BROJA_2PID','UIx')][i]=BROJA_2PID['UIY']/dt
        Information[('BROJA_2PID','UIv')][i]=BROJA_2PID['UIZ']/dt
        Information[('BROJA_2PID','CI')][i]=BROJA_2PID['CI']/dt
        

        MIxr=np.nansum(pxr*np.log2(pxr/px/pr))/dt
        MIvr=np.nansum(pvr*np.log2(pvr/pv/pr))/dt
        MIxvR=np.nansum(pxvr*np.log2(pxvr/pxv/pr))/dt
        PI_xR = np.nansum(pxr*np.log2(pxr/px/pr), axis = (0,1))
        PI_vR = np.nansum(pvr*np.log2(pvr/pv/pr), axis = (0,1))
        R = sum(np.minimum(PI_xR, PI_vR))/dt
        Information[('Beer','Red')][i] = R
        Information[('Beer','UIx')][i] = MIxr - R
        Information[('Beer','UIv')][i] = MIvr - R
        Information[('Beer','Syn')][i] = MIxvR - MIxr - MIvr + R

    return timeshift, Information

In [23]:
#calculate STA
xstate = np.random.randint(2, size=1000)
vstate = np.random.randint(2, size=1000)
rstate = vstate*2+xstate
dt = 1
window = [0,0] # second
timeshift, Information = PIfunc(rstate, xstate, vstate, dt, window)
# timeshift, A, MIxv, B, C, D = PIfunc(xstate, xstate, vstate, dt, window)


C:\Users\hydro_leo\anaconda3\lib\site-packages\ipykernel_launcher.py:61: RuntimeWarning: invalid value encountered in true_divide
C:\Users\hydro_leo\anaconda3\lib\site-packages\ipykernel_launcher.py:61: RuntimeWarning: divide by zero encountered in log2
C:\Users\hydro_leo\anaconda3\lib\site-packages\ipykernel_launcher.py:61: RuntimeWarning: invalid value encountered in multiply
C:\Users\hydro_leo\anaconda3\lib\site-packages\ipykernel_launcher.py:62: RuntimeWarning: invalid value encountered in true_divide
C:\Users\hydro_leo\anaconda3\lib\site-packages\ipykernel_launcher.py:62: RuntimeWarning: divide by zero encountered in log2
C:\Users\hydro_leo\anaconda3\lib\site-packages\ipykernel_launcher.py:62: RuntimeWarning: invalid value encountered in multiply
C:\Users\hydro_leo\anaconda3\lib\site-packages\ipykernel_launcher.py:72: RuntimeWarning: divide by zero encountered in log2
C:\Users\hydro_leo\anaconda3\lib\site-packages\ipykernel_launcher.py:72: RuntimeWarning: invalid value encountered

In [24]:
print(Information[('Beer','Syn')])
print(Information[('Beer','UIx')])
print(Information[('Beer','UIv')])
print(Information[('Beer','Red')])
print(Information[('Beer','Syn')]+Information[('Beer','UIx')]+Information[('Beer','UIv')]+Information[('Beer','Red')])

[0.9491623]
[0.04924257]
[0.04725256]
[0.94960296]
[1.9952604]


In [25]:
print(Information[('BROJA_2PID','CI')])
print(Information[('BROJA_2PID','UIx')])
print(Information[('BROJA_2PID','UIv')])
print(Information[('BROJA_2PID','SI')])
print(Information[('BROJA_2PID','CI')]+Information[('BROJA_2PID','UIx')]+Information[('BROJA_2PID','UIv')]+Information[('BROJA_2PID','SI')])

[0.]
[0.99838605]
[0.99639605]
[0.0004783]
[1.9952604]


In [26]:
# print(Information[('BROJA_2PID','CI')])
print(Information[('test','UIx')])
print(Information[('test','UIv')])
# print(Information[('BROJA_2PID','SI')])
# print(Information[('BROJA_2PID','CI')]+Information[('BROJA_2PID','UIx')]+Information[('BROJA_2PID','UIv')]+Information[('BROJA_2PID','SI')])

0.9964148681708521
0.9984048791790198


In [9]:
a = np.array([[[1, 2, 3],[4,5,6]], [[1, 2, 3],[4,5,6]]])
np.size(a, 0)

2

In [10]:
a

array([[[1, 2, 3],
        [4, 5, 6]],

       [[1, 2, 3],
        [4, 5, 6]]])

In [11]:
Histo3D2Dict(a)

{(0, 0, 0): 1.0,
 (1, 0, 0): 2.0,
 (2, 0, 0): 3.0,
 (0, 0, 1): 4.0,
 (1, 0, 1): 5.0,
 (2, 0, 1): 6.0,
 (0, 1, 0): 1.0,
 (1, 1, 0): 2.0,
 (2, 1, 0): 3.0,
 (0, 1, 1): 4.0,
 (1, 1, 1): 5.0,
 (2, 1, 1): 6.0}

In [12]:
PDF_series

NameError: name 'PDF_series' is not defined

In [ ]:
parms = dict()
parms['max_iters'] = 10
**parms

In [13]:
('a','b')

('a', 'b')